# Graph search algoritms sandbox

## Initial imports

In [67]:
import matplotlib.pyplot as plt
import collections
import heapq

### Simple function for creating graphs

In [21]:
class SimpleGraph:
    def __init__(self):
        self.edges = {}
    
    def neighbors(self, id):
        return self.edges[id]

example_graph = SimpleGraph()
example_graph.edges = {
    'A': ['B'],
    'B': ['A', 'C', 'D'],
    'C': ['A'],
    'D': ['E', 'A'],
    'E': ['B']
}


In [24]:
class Queue:
    def __init__(self):
        self.elements = collections.deque()
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, x):
        self.elements.append(x)
    
    def get(self):
        return self.elements.popleft()

### Breadth First Search

In [57]:
def breadth_first_search_1(graph, start):
        # print out what we find
        frontier = Queue()
        frontier.put(start)
        visited = {}
        visited[start] = True
        while not frontier.empty():
            current = frontier.get()
            print("Visiting %r" % current)
            for next in graph.neighbors(current):
                if next not in visited:
                    frontier.put(next)
                    visited[next] = True

breadth_first_search_1(example_graph, 'B')

Visiting 'B'
Visiting 'A'
Visiting 'C'
Visiting 'D'
Visiting 'E'


### Handle more complicated graphs

In [46]:
# Create Grid that stores tuples instead of 
class SquareGrid:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.walls = []
    
    def in_bounds(self, id):
        (x, y) = id
        return 0 <= x < self.width and 0 <= y < self.height
    
    def passable(self, id):
        return id not in self.walls
    
    def neighbors(self, id):
        (x, y) = id
        results = [(x+1, y), (x, y-1), (x-1, y), (x, y+1)]
        if (x + y) % 2 == 0: results.reverse() # aesthetics
        results = filter(self.in_bounds, results)
        results = filter(self.passable, results)
        return results

In [74]:
# Utility functions for dealing with square grids
def from_id_width(id, width):
    return (id % width, id // width)

def draw_tile(graph, id, style, width):
    r = "."
    if 'number' in style and id in style['number']: r = "%d" % style['number'][id]
    if 'point_to' in style and style['point_to'].get(id, None) is not None:
        (x1, y1) = id
        (x2, y2) = style['point_to'][id]
        if x2 == x1 + 1: r = "\u2192"
        if x2 == x1 - 1: r = "\u2190"
        if y2 == y1 + 1: r = "\u2193"
        if y2 == y1 - 1: r = "\u2191"
    if 'start' in style and id == style['start']: r = "A"
    if 'goal' in style and id == style['goal']: r = "Z"
    if 'path' in style and id in style['path']: r = "@"
    if id in graph.walls: r = "#" * width
    return r

def draw_grid(graph, width=2, **style):
    for y in range(graph.height):
        for x in range(graph.width):
            print("%%-%ds" % width % draw_tile(graph, (x, y), style, width), end="")
        print()

DIAGRAM1_WALLS = [from_id_width(id, width=30) for id in [21,22,51,52,81,82,93,94,111,112,123,124,133,134,141,142,153,154,163,164,171,172,173,174,175,183,184,193,194,201,202,203,204,205,213,214,223,224,243,244,253,254,273,274,283,284,303,304,313,314,333,334,343,344,373,374,403,404,433,434]]


In [75]:
g = SquareGrid(30, 15)
g.walls = DIAGRAM1_WALLS # long list, [(21, 0), (21, 2), ...]
draw_grid(g)

. . . . . . . . . . . . . . . . . . . . . ####. . . . . . . 
. . . . . . . . . . . . . . . . . . . . . ####. . . . . . . 
. . . . . . . . . . . . . . . . . . . . . ####. . . . . . . 
. . . ####. . . . . . . . . . . . . . . . ####. . . . . . . 
. . . ####. . . . . . . . ####. . . . . . ####. . . . . . . 
. . . ####. . . . . . . . ####. . . . . . ##########. . . . 
. . . ####. . . . . . . . ####. . . . . . ##########. . . . 
. . . ####. . . . . . . . ####. . . . . . . . . . . . . . . 
. . . ####. . . . . . . . ####. . . . . . . . . . . . . . . 
. . . ####. . . . . . . . ####. . . . . . . . . . . . . . . 
. . . ####. . . . . . . . ####. . . . . . . . . . . . . . . 
. . . ####. . . . . . . . ####. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . ####. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . ####. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . ####. . . . . . . . . . . . . . . 


In [65]:
def early_exit(graph, start, goal):
    frontier = Queue()
    frontier.put(start)
    came_from = {}
    came_from[start] = None

    while not frontier.empty():
        current = frontier.get()
        
        # Early exit thing
        if current == goal: 
            break           
        
        for next in graph.neighbors(current):
            if next not in came_from:
                frontier.put(next)
                came_from[next] = current
                
    return came_from
    g = SquareGrid(30, 15)
    g.walls = DIAGRAM1_WALLS


# Start Point and Goal points
_start = (7, 2)
_goal = (17, 2)

parents = early_exit(g, _start, _goal )
draw_grid(g, width=2, point_to=parents, start=_start, goal=_goal)

→ → → → → → ↓ ↓ ↓ ↓ ← ← ← ← ← ← ← . . . . ####. . . . . . . 
→ → → → → → → ↓ ↓ ← ← ← ← ← ← ← ← ← . . . ####. . . . . . . 
→ → → → → → → A ← ← ← ← ← ← ← ← ← Z . . . ####. . . . . . . 
→ → ↑ ####→ ↑ ↑ ↑ ← ← ← ← ← ← ← ← ← . . . ####. . . . . . . 
→ ↑ ↑ ####↑ ↑ ↑ ↑ ↑ ← ← ← ####↑ ← . . . . ####. . . . . . . 
↑ ↑ ↑ ####↑ ↑ ↑ ↑ ↑ ↑ ← ← ####↑ . . . . . ##########. . . . 
. ↑ ↑ ####↑ ↑ ↑ ↑ ↑ ↑ ↑ ← ####. . . . . . ##########. . . . 
. . ↑ ####↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ####. . . . . . . . . . . . . . . 
. . . ####↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ####. . . . . . . . . . . . . . . 
. . . ####↑ ↑ ↑ ↑ ↑ ↑ ↑ . ####. . . . . . . . . . . . . . . 
. . . ####↑ ↑ ↑ ↑ ↑ ↑ . . ####. . . . . . . . . . . . . . . 
. . . ####. ↑ ↑ ↑ ↑ . . . ####. . . . . . . . . . . . . . . 
. . . . . . . ↑ ↑ . . . . ####. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . ####. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . ####. . . . . . . . . . . . . . . 


## Dijkstra 


In [68]:
# Now we need to implement also weights to escalate cost of movement
class GridWithWeights(SquareGrid):
    
    def __init__(self, width, height):
        super().__init__(width, height)
        self.weights = {}
        
    # this function tells us the cost of moving
    def cost(self, from_node, to_node):
        return self.weights.get(to_node, 1)

diagram4 = GridWithWeights(10, 10)
diagram4.walls = [(1, 7), (1, 8), (2, 7), (2, 8), (3, 7), (3, 8)]
diagram4.weights = {loc: 5 for loc in [(3, 4), (3, 5), (4, 1), (4, 2),
                                       (4, 3), (4, 4), (4, 5), (4, 6), 
                                       (4, 7), (4, 8), (5, 1), (5, 2),
                                       (5, 3), (5, 4), (5, 5), (5, 6), 
                                       (5, 7), (5, 8), (6, 2), (6, 3), 
                                       (6, 4), (6, 5), (6, 6), (6, 7), 
                                       (7, 3), (7, 4), (7, 5)]}


class PriorityQueue:
    def __init__(self):
        self.elements = []
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self):
        return heapq.heappop(self.elements)[1]

# Sometimes this variant is called Uniform Cost Search
def dijkstra_search(graph, start, goal):
    
    # Good thing is that we do not put all the nodes at the start
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0

    while not frontier.empty():
        current = frontier.get()
        
        # In case to not insert all nodes I implemented EarlyExit also
        if current == goal:
            break

        for next in graph.neighbors(current):
            new_cost = cost_so_far[current] + graph.cost(current, next)
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost
                frontier.put(next, priority)
                came_from[next] = current
                
    return came_from, cost_so_far

def reconstruct_path(came_from, start, goal):
    current = goal
    path = [current]
    while current != start:
        current = came_from[current]
        path.append(current)
    path.append(start)
    path.reverse() 
    return path

In [70]:
# Test time
came_from, cost_so_far = dijkstra_search(diagram4, (1, 4), (7, 8))
draw_grid(diagram4, width=3, point_to=came_from, start=(1, 4), goal=(7, 8))
print()
draw_grid(diagram4, width=3, number=cost_so_far, start=(1, 4), goal=(7, 8))
print()
draw_grid(diagram4, width=3, path=reconstruct_path(came_from, start=(1, 4), goal=(7, 8)))

↓  ↓  ←  ←  ←  ←  ←  ←  ←  ←  
↓  ↓  ←  ←  ←  ↑  ↑  ←  ←  ←  
↓  ↓  ←  ←  ←  ←  ↑  ↑  ←  ←  
↓  ↓  ←  ←  ←  ←  ←  ↑  ↑  .  
→  A  ←  ←  ←  ←  .  .  .  .  
↑  ↑  ←  ←  ←  ←  .  .  .  .  
↑  ↑  ←  ←  ←  ←  ←  .  .  .  
↑  #########↑  ←  ↓  .  .  .  
↑  #########↓  ↓  ↓  Z  .  .  
↑  ←  ←  ←  ←  ←  ←  ←  ←  .  

5  4  5  6  7  8  9  10 11 12 
4  3  4  5  10 13 10 11 12 13 
3  2  3  4  9  14 15 12 13 14 
2  1  2  3  8  13 18 17 14 .  
1  A  1  6  11 16 .  .  .  .  
2  1  2  7  12 17 .  .  .  .  
3  2  3  4  9  14 19 .  .  .  
4  #########14 19 18 .  .  .  
5  #########15 16 13 Z  .  .  
6  7  8  9  10 11 12 13 14 .  

.  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  
@  @  .  .  .  .  .  .  .  .  
@  .  .  .  .  .  .  .  .  .  
@  .  .  .  .  .  .  .  .  .  
@  #########.  .  .  .  .  .  
@  #########.  .  @  @  .  .  
@  @  @  @  @  @  @  .  .  .  


## A*

### Create Heuristics

In [76]:
def heuristic(a, b):
    (x1, y1) = a
    (x2, y2) = b
    return abs(x1 - x2) + abs(y1 - y2)

In [87]:
## Heuristics search (Greedy Best First Search)
def greedy_best_first_search(graph, start, goal):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    came_from[start] = None

    while not frontier.empty():
        current = frontier.get()

        if current == goal:
            break

        for next in graph.neighbors(current):
            if next not in came_from:
                # We do not use cost_so_far from Dijsktra
                priority = heuristic(goal, next)
                frontier.put(next, priority)
                came_from[next] = current
                
    return cost_so_far

In [88]:
cost_so_far = greedy_best_first_search(diagram4, (1, 4), (7, 8))
draw_grid(diagram4, width=3, point_to=came_from, start=(1, 4), goal=(7, 8))
print()
draw_grid(diagram4, width=3, number=cost_so_far, start=(1, 4), goal=(7, 8))
print()
draw_grid(diagram4, width=3, path=reconstruct_path(came_from, start=(1, 4), goal=(7, 8)))

.  .  .  .  .  .  .  .  .  .  
.  ↓  ↓  ↓  .  .  .  .  .  .  
↓  ↓  ↓  ↓  ←  .  .  .  .  .  
↓  ↓  ↓  ←  ←  .  .  .  .  .  
→  A  ←  ←  ←  .  .  .  .  .  
→  ↑  ←  ←  ←  .  .  .  .  .  
→  ↑  ←  ←  ←  ←  .  .  .  .  
↑  #########↑  .  ↓  .  .  .  
↑  #########↓  ↓  ↓  Z  .  .  
↑  ←  ←  ←  ←  ←  ←  ←  .  .  

.  .  .  .  .  .  .  .  .  .  
.  3  4  5  .  .  .  .  .  .  
3  2  3  4  9  .  .  .  .  .  
2  1  2  3  8  .  .  .  .  .  
1  A  1  6  11 .  .  .  .  .  
2  1  2  7  12 .  .  .  .  .  
3  2  3  4  9  14 .  .  .  .  
4  #########14 .  18 .  .  .  
5  #########15 16 13 Z  .  .  
6  7  8  9  10 11 12 13 .  .  

.  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  
.  @  .  .  .  .  .  .  .  .  
.  @  .  .  .  .  .  .  .  .  
@  @  .  .  .  .  .  .  .  .  
@  #########.  .  .  .  .  .  
@  #########.  .  @  @  .  .  
@  @  @  @  @  @  @  .  .  .  


### A-star search algorithm

In [77]:
def a_star_search(graph, start, goal):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        current = frontier.get()
        
        if current == goal:
            break
        
        for next in graph.neighbors(current):
            new_cost = cost_so_far[current] + graph.cost(current, next)
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(goal, next)
                frontier.put(next, priority)
                came_from[next] = current
    
    return came_from, cost_so_far

In [85]:
came_from, cost_so_far = a_star_search(diagram4, (1, 4), (7, 8))
draw_grid(diagram4, width=3, point_to=came_from, start=(1, 4), goal=(7, 8))
print()
draw_grid(diagram4, width=3, number=cost_so_far, start=(1, 4), goal=(7, 8))

.  .  .  .  .  .  .  .  .  .  
.  ↓  ↓  ↓  .  .  .  .  .  .  
↓  ↓  ↓  ↓  ←  .  .  .  .  .  
↓  ↓  ↓  ←  ←  .  .  .  .  .  
→  A  ←  ←  ←  .  .  .  .  .  
→  ↑  ←  ←  ←  .  .  .  .  .  
→  ↑  ←  ←  ←  ←  .  .  .  .  
↑  #########↑  .  ↓  .  .  .  
↑  #########↓  ↓  ↓  Z  .  .  
↑  ←  ←  ←  ←  ←  ←  ←  .  .  

.  .  .  .  .  .  .  .  .  .  
.  3  4  5  .  .  .  .  .  .  
3  2  3  4  9  .  .  .  .  .  
2  1  2  3  8  .  .  .  .  .  
1  A  1  6  11 .  .  .  .  .  
2  1  2  7  12 .  .  .  .  .  
3  2  3  4  9  14 .  .  .  .  
4  #########14 .  18 .  .  .  
5  #########15 16 13 Z  .  .  
6  7  8  9  10 11 12 13 .  .  
